In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import os

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

import tensorflow as tf
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from loop import TrainingLoop

In [2]:
data = pd.read_csv('data/spirals.csv')
X = data.drop('label', axis=1).to_numpy()
y = data['label'].to_numpy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

In [14]:
tf.random.set_seed(42)
np.random.seed(42)

model = Sequential()
model.add(Dense(7, activation='sigmoid'))
model.add(Dense(7, activation='sigmoid'))
model.add(Dense(1, activation='sigmoid'))

#model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#model.fit(X_train, y_train, batch_size=16, epochs=100)

In [5]:
optimizer = tf.keras.optimizers.Adam()
loss_function = tf.keras.losses.BinaryCrossentropy()

batch_size = 16
epochs = 150

train_metrics = tf.keras.metrics.CategoricalAccuracy()
val_metrics = tf.keras.metrics.CategoricalAccuracy()

In [4]:
@tf.function
def calc_loss(x_train, y_train):
    with tf.GradientTape() as tape:
        logits = model(x_train, training=False)
        loss_value = loss_function(y_train, logits)
    return loss_value

In [6]:
def batch_selector(data, idx):
    largest_loss = 0
    lergest_loss_idx = idx
    for i in range(idx, idx+5):
        x_batch_train = train_data[i][0]
        y_batch_train = train_data[i][1]
        loss = calc_loss(x_batch_train, y_batch_train)
        if loss > largest_loss:
            largest_loss = loss
            largest_loss_idx = i
    return train_data[largest_loss_idx][0], train_data[largest_loss_idx][1], largest_loss

In [7]:
training = TrainingLoop(model, X_train, y_train, loss_function, optimizer, validation_split=0.1, batch_size=batch_size)
training.train(epochs)

	  0% | 0/45 [00:00<?, ?it/s]WARNING:tensorflow:Layer dense is casting an input tensor from dtype float64 to the layer's dtype of float32, which is new behavior in TensorFlow 2.  The layer has dtype float32 because its dtype defaults to floatx.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch 1/150	Loss: 0.6965: 	100% | 45/45 [00:02<00:00, 20.55it/s]
Epoch 2/150	Loss: 0.6959: 	100% | 45/45 [00:00<00:00, 87.89it/s]
Epoch 3/150	Loss: 0.6955: 	100% | 45/45 [00:00<00:00, 92.98it/s]
Epoch 4/150	Loss: 0.6951: 	100% | 45/45 [00:00<00:00, 104.89it/s]
Ep

In [8]:
pred = model.predict(X_test)
accuracy_score(y_test, pred.round())

0.965

In [13]:
losses = []
def batch_sorting(data, idx):
    global losses
    if idx == 0:
        for i in range(len(data)):
            x_batch_train = data[i][0]
            y_batch_train = data[i][1]
            losses.append(calc_loss(x_batch_train, y_batch_train))
    largest_loss = max(losses)
    largest_loss_idx = losses.index(largest_loss)
    losses[largest_loss_idx] = 0
    print('batch selector:\tidx:', str(largest_loss_idx).ljust(5), 'loss:', largest_loss.numpy())
    if idx == len(data)-1:
        losses.clear()
        print("end of epoch\n")
    return largest_loss_idx

In [15]:
training = TrainingLoop(model, X_train, y_train, loss_function, optimizer, validation_split=0.1, batch_size=batch_size, batch_selection=batch_sorting)
training.train(3)

	  0% | 0/45 [00:00<?, ?it/s]batch selector:	idx: 21    loss: 0.26195067

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch 1/3	Loss: 0.6829: 	 13% | 6/45 [00:01<00:38,  1.01it/s]batch selector:	idx: 25    loss: 0.23996213
batch selector:	idx: 18    loss: 0.22623077
batch selector:	idx: 4     loss: 0.21983002
batch selector:	idx: 28    loss: 0.19484635
batch selector:	idx: 38    loss: 0.18960327
batch selector:	idx: 23    loss: 0.18660906
batch selector:	idx: 14    loss: 0.16490006
batch selector:	idx: 40    loss: 0.16423082
batch selector:	idx: 8